* 색인 & 가중치 => 학습
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

* 질의 -> 색인과정의 2 to 3
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력

## 1. 모든 문서들에 대해 토큰화 수행하기

In [54]:
# 이 코드에서는 지금까지 배운 내용들을 활용해 텍스트 문서에서 쿼리 단어를 검색한 후 해당 쿼리가 위치한 문서 경로를 출력해 봅니다. 

def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [55]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [56]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [57]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|"
                          r"한국경제|재배포|기사제보|보도자료|한경닷컴|네이버에서|자세히보기|자세히 보기|모바일한경|구독신청|"
                          r"머니투데이|돈이 보이는 리얼타임 뉴스|연예뉴스|파이낸셜뉴스|무단 전재 금지") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    
    return patternList

In [58]:
from os import listdir

FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
NewsContent = list() 

for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180


In [59]:
patternList = getPatternList()
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]: 
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[0]) # 불용어 처리가 모두 끝난 뉴스기사 목록


   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다 당시 호소문에는 전세계 저명한 학술

In [60]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()

# 단일 뉴스 기사를 토큰화해주는 함수
def Tokenizer(NewsContent):
    dictTerm = list()
    dictPos = list()
    dictNoun = list()
    dictNgram = list()
    th = 1 
    
    for sentence in sent_tokenize(NewsContent):
        for token in word_tokenize(sentence):
            if len(token) > th:
                dictTerm.append(token)
                # 아래서부터는 list이기때문에 extend를 사용해야 함. 
                dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
                dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
                dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
                
    # 빠른 속도 및 중복 데이터 처리를 위해 set사용 (유일한 단어만 남김)
    dictTerm = list(set(dictTerm))
    dictPos = list(set(dictPos))
    dictNoun = list(set(dictNoun))
    dictNgram = list(set(dictNgram))
        
    return dictTerm, dictPos, dictNoun, dictNgram

In [61]:
# TokenizedNewsContent = list()
# 
# for i in range(len(FileList)):
#     TokenizedNewsContent.append(Tokenizer(NewsContent[i]))

In [62]:
# # len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram) of 0th txt file
# print(len(TokenizedNewsContent[0][0]), len(TokenizedNewsContent[0][1]), len(TokenizedNewsContent[0][2]), len(TokenizedNewsContent[0][3]))
# 
# # dictTerm, dictPos, dictNoun, dictNgram 테스트용
# # for i in range(len(FileList)):
# #     print(str(i) + "th txt : " + str(len(list(set(TokenizedNewsContent[i][0] + TokenizedNewsContent[i][1] + TokenizedNewsContent[i][2] + TokenizedNewsContent[i][3])))))

In [63]:
# # 0 : dictTerm, 1 : dictPos, 2 : dictNoun, 3 : dictNgram
# print(TokenizedNewsContent[0][0])

## 2. TDF-ID 방식으로 검색 수행해보기
WIP(2019-03-29 10:28 PM) 



In [69]:
collection = [] 
for i in range(len(NewsContent)):
    collection.append((str(FileList[i]), NewsContent[i])) # (파일경로, 불용어 처리 완료된 기사) 튜플 구성 
print(collection[0])

# in-memory (Hash Key 값)
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치, 단어2:포스팅위치, ...}
queryPosting = dict()

# 전체 문서 목록(Dictionary)
# [0:문서1, 1:문서2, ...]
globalDocument = list()

# disk
# 사전에 있는 색인어 중, 어느 문서에서, 몇 번 나타났는지
# [(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# [0:Tuple(lexiconIdx, documentIdx, freq, 다음포스팅위치-fptr)]
# 메모리 X, File OK
globalPosting = list()

('C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//IT과학-0000003610.txt', '   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 

In [70]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # Local 작업
    for term in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if term not in localPosting.keys():
            localPosting[term] = 1
        # 있으면, 빈도 증가
        else:
            localPosting[term] += 1
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in queryPosting.keys(): 
            lexiconIdx = len(queryPosting)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx, docIdx, termFreq, -1)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            lexiconIdx = list(queryPosting.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = queryPosting[indexTerm]
            postingData = (lexiconIdx, docIdx, termFreq, beforeIdx)
            globalPosting.append(postingData)
            queryPosting[indexTerm] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [72]:
# queryPosting, globalDocument

In [74]:
# globalPosting

In [76]:
for indexTerm, postingIdx in queryPosting.items():
    # indexTerm:단어: postingIdx:위치, ...
    # print(indexTerm)
    
    while True: # Posting Next:-1
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        # print('  DocName:{0} - TermFreq:{1} - Next:{2}'.format(globalDocument[postingData[1]], postingData[2], postingData[3]))
        postingIdx = postingData[3]

In [77]:
globalPosting[queryPosting['이어폰']]

(775, 20, 2, 4244)

In [78]:
globalPosting[globalPosting[queryPosting['이어폰']][3]]   # 다음 주소가 "-1" 일때 까지 반복해서 찾음

(775, 15, 2, 810)

## 3. TF-IDF 방식과 cosine simmilarity으로 검색해보기

In [95]:
from math import log10

# TF 구하기
def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0

def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1+log10(freq)
    else:
        return 0

def doubleNormalTF(K, freq, maxFreq): 
    return K + ((1-K) * (freq/maxFreq))

In [96]:
# IDF(Inverse Document Frequency 구하기
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothigIDF(N, df):
    return log10((N+1)/df)

def probabilityIDF(N, df):
    return log10((N-df+1)/df)

In [97]:
def innerProduct(x, y): # 내적 연산
    return x * y

In [98]:
globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    maxFreq = max(localPosting.values())
    
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys():
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting)
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
        else:
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, termFreq, maxFreq), beforeIdx]
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx

In [100]:
# globalPosting

In [101]:
N = len(globalDocument)
globalLexiconIDF = dict()
globalDocumentLength = dict()

for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm : 단어, postingIdx :위치(포인터역할), ...
    df = 0
    oldPostingIdx = postingIdx
    
    while True: # ptr Next: -1 
        if postingIdx == -1:
            break
        
        df += 1
        postingData = globalPosting[postingIdx]
        postingIdx = postingData[3]
        
    idf = smoothigIDF(N, df)
    globalLexiconIDF[indexTerm] = idf # 나중에 단어가 나왔을때 idf를 불러올 수 있도록 하기 위함
    postingIdx = oldPostingIdx
    # print('단어 : {0} / IDF : {1}'.format(indexTerm, idf))
    
    while True:
        if postingIdx == -1:
            break
        
        postingData = globalPosting[postingIdx]
        TF = postingData[2]
        globalPosting[postingIdx][2] *= idf
        # print('    Documents:{0} / TF:{1} / TF-IDF:{2}'.format(globalDocument[postingData[1]], 
        #                                                        TF, globalPosting[postingIdx][2]))
        
        postingIdx = postingData[3]
        
        # 색인하는 과정에서 사실상 검색까지 모두 한 셈이 되므로 색인 과정에만 시간이 들어 굉장히 빠른 연산속도를 보임. 
        # 키 에러 방지용(키값이 있다면 제곱해서 누적해주면 됨)
        if postingData[1] not in globalDocumentLength.keys():
            globalDocumentLength[postingData[1]] = globalPosting[postingIdx][2] ** 2
        else:
            globalDocumentLength[postingData[1]] += globalPosting[postingIdx][2] ** 2

In [102]:
query = '이어폰 과 스마트폰' # Document

queryPosting = dict()

for term in query.lower().split():
    if term not in queryPosting.keys():
        queryPosting[term] = 1
    else:
        queryPosting[term] += 1
        
maxFreq = max(queryPosting.values())

for indexTerm, termFreq in queryPosting.items():
    queryPosting[indexTerm] = doubleNormalTF(0.5, termFreq, maxFreq)

In [103]:
queryPosting

{'이어폰': 1.0, '과': 1.0, '스마트폰': 1.0}

In [110]:
candidateList = dict()

for indexTerm, queryWeight in queryPosting.items(): # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
    if indexTerm in globalLexicon.keys():
        postingIdx = globalLexicon[indexTerm]
        
        while True: # ptr Next: -1
            if postingIdx == -1:
                break
            
            postingData = globalPosting[postingIdx]
            postingIdx = postingData[3]
            documentWeight = postingData[2]
            
            
            # 키 에러 방지용
            if postingData[1] not in candidateList.keys():
                candidateList[postingData[1]] = innerProduct(queryWeight, documentWeight)
            else:
                candidateList[postingData[1]] += innerProduct(queryWeight, documentWeight)

for documentIdx, sumProduct in candidateList.items():
    candidateList[documentIdx] /= globalDocumentLength[documentIdx]

In [118]:
# 정렬 (reversed = False는 작은값부터 큰 값 순서)
resultList = sorted(candidateList.items(), key=lambda x:x[1], reverse=True)

print('query: ', query)

for i, (documentIdx, distance) in enumerate(resultList):
    print('순위:{0} / 문서:{1} / 유사도:{2}'.format((i+1), globalDocument[documentIdx], distance))
    print('   document:{0}'.format(collection[documentIdx][1]))

query:  이어폰 과 스마트폰
순위:1 / 문서:C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts//IT과학-0000003610.txt / 유사도:0.2513430003394383
   document:   오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원   1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과   등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가

## TDM-DTM-TWM

In [119]:
from konlpy.corpus import kobill
from collections import defaultdict
from konlpy.tag import Kkma

ma = Kkma().morphs

In [122]:
def getDocReprByDefaultDict():
    docRepr = defaultdict(lambda: defaultdict(int))
    
    # collection
    for docName in collection:
        for token in str(collection).split(): 
            for morpheme in ma(token):
                docRepr[docName][morpheme] += 1
    
    return docRepr

In [123]:
DTM = getDocReprByDefaultDict()